In [13]:
try:
    import urllib.request as urllib2
except ImportError:
    import urllib2
from bs4 import BeautifulSoup
import sys

In [22]:
def getAllDoxyDonkeyPosts(url,links):
    request=urllib2.Request(url)
    response=urllib2.urlopen(request)
    soup=BeautifulSoup(response)
    #print(soup)
    #print(soup.find_all('a').text)
    for a in soup.findAll('a'):
        try:
            if a.has_attr('href') and a.has_attr('title'):
                url=a['href']
                title=a['title']
                if title=='Older Posts':
                    print (title, url)
                    links.append(url)
                    getAllDoxyDonkeyPosts(url,links)
        except Exception as e:
            title= " "
            print("exception",e.__class__)
            
    #print(" outside for::",len(links))
    return
            
            

In [23]:
links=[]
links.append("https://doxydonkey.blogspot.com/")
getAllDoxyDonkeyPosts("https://doxydonkey.blogspot.com/",links)

Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-05-23T19:53:00-07:00&max-results=7
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-05-14T19:02:00-07:00&max-results=7&start=7&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-05-02T19:43:00-07:00&max-results=7&start=14&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-04-17T19:26:00-07:00&max-results=7&start=21&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-04-10T18:56:00-07:00&max-results=7&start=28&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-03-30T19:57:00-07:00&max-results=7&start=35&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-03-20T19:47:00-07:00&max-results=7&start=42&by-date=false
Older Posts https://doxydonkey.blogspot.com/search?updated-max=2017-03-02T17:42:00-08:00&max-results=7&start=49&by-date=false
Older Posts http

In [46]:
def getAllDoxyDonkeyText(testUrl):
    request=urllib2.Request(testUrl)
    response=urllib2.urlopen(request)
    soup=BeautifulSoup(response)
    
    myDivs=soup.findAll('div',{"class":'post-body entry-content'})
    posts=[]
    
    for div in myDivs:
        posts+=map(lambda p:p.text, div.findAll("li"))
    #print(len(posts))
    return posts

In [47]:
#post=getAllDoxyDonkeyText("https://doxydonkey.blogspot.com/")
doxyDonkeyPosts=[]

In [48]:
for link in links:
    doxyDonkeyPosts+=getAllDoxyDonkeyText(link)  

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [51]:
vectorizer=TfidfVectorizer(max_df=0.5,min_df=2,stop_words='english')
X=vectorizer.fit_transform(doxyDonkeyPosts)
X

<2804x13219 sparse matrix of type '<class 'numpy.float64'>'
	with 280830 stored elements in Compressed Sparse Row format>

In [54]:
#print(X[0])
from sklearn.cluster import KMeans

In [56]:
km=KMeans(n_clusters=3, init="k-means++", max_iter=100, n_init=1, verbose= True)
km.fit(X)

Initialization complete
Iteration 0, inertia 5162.718162381645
Iteration 1, inertia 2678.0858568118015
Iteration 2, inertia 2673.936593591847
Iteration 3, inertia 2672.990323874513
Iteration 4, inertia 2672.6333704402587
Iteration 5, inertia 2672.4095259412675
Iteration 6, inertia 2672.189657964154
Iteration 7, inertia 2671.78450475882
Iteration 8, inertia 2671.263431071896
Iteration 9, inertia 2670.266210120417
Iteration 10, inertia 2669.4279561021112
Iteration 11, inertia 2669.062297348179
Iteration 12, inertia 2668.851637840718
Iteration 13, inertia 2668.706002672006
Iteration 14, inertia 2668.5623134920293
Iteration 15, inertia 2668.44945586
Iteration 16, inertia 2668.3533387749967
Iteration 17, inertia 2668.263654451316
Iteration 18, inertia 2668.1744918412255
Iteration 19, inertia 2668.1019224391384
Iteration 20, inertia 2668.058588508997
Iteration 21, inertia 2668.0366417350087
Iteration 22, inertia 2668.01911006297
Iteration 23, inertia 2667.9907358805544
Iteration 24, inertia 

KMeans(max_iter=100, n_clusters=3, n_init=1, verbose=True)

In [57]:
import numpy as np
np.unique(km.labels_,return_counts=True)

(array([0, 1, 2]), array([1822,  390,  592], dtype=int64))

In [67]:
text={}
#print(len(doxyDonkeyPosts))
for i , cluster in enumerate (km.labels_):
    doc=doxyDonkeyPosts[i]
    if cluster not in text.keys():
        text[cluster]=doc
    else:
        text[cluster]+=doc 

In [68]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from collections import defaultdict
from string import punctuation
from heapq import nlargest
import nltk

In [69]:
_stopwords=set(stopwords.words('english')+list(punctuation)+["million","billion","millions","billions","'s","y/y","''"])

In [70]:
keywords={}
counts={}
print(text)
for cluster in range(3):
    word_sent=word_tokenize(text[cluster].lower())
    word_sent=[word for word in word_sent if word not in _stopwords]
    freq=FreqDist(word_sent)
    keywords[cluster]=nlargest(100, freq,key=freq.get)
    counts[cluster]=freq

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [72]:
unique_keywords={}
for cluster in range(3):
    other_cluster=list(set(range(3))-set([cluster]))
    other_keywords=set(keywords[other_cluster[0]]).union(set(keywords[other_cluster[1]]))
    unique=set(keywords[cluster])-other_keywords
    unique_keywords[cluster]=nlargest(10, unique,key=counts[cluster].get)
print(unique_keywords)

{0: ['uber', 'india', 'customers', 'chinese', 'deal', 'firm', 'public', 'capital', 'round', 'price'], 1: ['quarter', 'profit', 'rose', 'earnings', 'analysts', 'cents', 'net', 'fell', 'reported', 'trading'], 2: ['ads', 'video', 'ad', 'search', 'apps', 'social', 'snapchat', 'youtube', 'advertisers', 'content']}


In [73]:
#to classify new article use KNN
article="Amazon.com Inc.’s shares rose briefly just above $1,000 for the first time Tuesday, marking a new milestone for a company wooing investors by dominating online commerce and cloud computing. Amazon hit an intraday high of $1,001.20 in New York before ending the day little changed at $996.70. The stock is up almost 40 percent from a year ago and more than double the 15 percent gain of the S&P 500 Index in the same period. Investors are thinking about how much further Amazon can grow as it tries to replicate its U.S. success abroad. The Seattle company’s $478 billion market value is double that of Wal-Mart Stores Inc. even though the world’s biggest retailer will have sales three times larger than Amazon’s this year. Investors put more value in Amazon’s web traffic and delivery network than they do in Wal-Mart’s vast store presence because online spending will grow more than four times faster than overall retail spending this year as shoppers continue to shift from stores to websites, according to EMarketer Inc. The world’s largest online retailer is dominating e-commerce with its $99-a-year Amazon Prime subscription, which includes delivery discounts, music and video streaming and photo storage that keep shoppers engaged with the website. Another Amazon advantage is its profitable and fast-growing cloud-computing division Amazon Web Services, which maintains a global network of data centers and rents out storage space and computing functions to clients in a variety of industries, including Netflix Inc. and Airbnb Inc. as well as Capital One Financial Corp. and the federal government."

In [75]:
from sklearn.neighbors import KNeighborsClassifier
classifier= KNeighborsClassifier()
classifier.fit(X,km.labels_)

KNeighborsClassifier()

In [78]:
test=vectorizer.transform([article])
test

<1x13219 sparse matrix of type '<class 'numpy.float64'>'
	with 113 stored elements in Compressed Sparse Row format>

In [79]:
classifier.predict(test)

array([0])